In [1]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.6/661.6 kB 13.4 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch_geometric: filename=torch_geometric-2.3.1-py3-none-any.whl size=910459 sha256=b140590f97b5a32637bb66b88a37f33d92f778069a92823c498c89de44e27ce6
  Stored in directory: /root/.cache/pip/wheels/ac/dc/30/e2874821ff308ee67dcd7a66dbde912411e19e35a1addda028
Successfully built torch_geometric


In [16]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained('google/flan-t5-base')
model = T5ForConditionalGeneration.from_pretrained('google/flan-t5-base')
input_ids = tokenizer.encode("answer: hello, how are you", return_tensors="pt")  # Batch size 1
outputs = model.generate(input_ids)
print([tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in outputs])

['good']


In [ ]:
import torch
import torch.nn as nn
from torch_geometric.nn import GCNConv
from transformers import T5Model, T5TokenizerFast
import networkx as nx
from torch_geometric.data import Data
import xml.etree.ElementTree as ET
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from torch_geometric.data import Batch
from transformers import T5Model
from torch_geometric.nn import GCNConv
from datasets import load_dataset
from torch_geometric.data import Data
from transformers import T5TokenizerFast
import networkx as nx
import torch
import re
import matplotlib.pyplot as plt

#model_name = "t5-small"
model_name = "google/flan-t5-small"


class WebNLGDataset(Dataset):
    def __init__(self, dataset, max_edges=512):
        self.dataset = dataset
        self.tokenizer = T5TokenizerFast.from_pretrained(model_name)
        self.node_to_idx = {}  # Node to index mapping
        self.max_edges = max_edges

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        data_dict = self.dataset[idx]
        text = data_dict['lex']['text'][0]
        triples = data_dict['original_triple_sets']['otriple_set'][0]

        graph_nx = self.triples_to_graph(triples)
        edge_index = self.get_edge_index(graph_nx)

        encoding = self.tokenizer.encode_plus(text, truncation=True, padding='max_length', max_length=512, return_tensors='pt')

        graph_data = Data(x=encoding['input_ids'].squeeze(dim=0), edge_index=edge_index)
        graph_data.attention_mask = encoding['attention_mask'].squeeze(dim=0)
        graph_data.y = encoding['input_ids'].squeeze(dim=0)
        
        #print("Original Sample: \n", text, "\n", triples)  # Print the original sample      
        #print("NetworkX Graph: \n", graph_nx.edges) # Print the NetworkX graph
        #self.visualize_graph(graph_nx)

        return graph_data

    def triples_to_graph(self, triples):
        self.node_to_idx = {}  # reset for each new graph
        graph_nx = nx.MultiDiGraph()
        for triple in triples:
            triple = re.sub(r'\([^)]*\)', '', triple).split('|')  # remove brackets and split by '|'
            subject, relation, obj = map(str.strip, triple)

            # Add string node names to the graph
            if subject not in self.node_to_idx:
                self.node_to_idx[subject] = len(self.node_to_idx)
            if obj not in self.node_to_idx:
                self.node_to_idx[obj] = len(self.node_to_idx)

            graph_nx.add_edge(subject, obj, key=relation)
        return graph_nx

    def get_edge_index(self, graph_nx):
        edge_index = torch.tensor([[self.node_to_idx[n] for n in edge[:2]] for edge in graph_nx.edges]).t().contiguous()
        return edge_index
    
    def visualize_graph(self, graph_nx):
        plt.figure(figsize=(8, 6))
        pos = nx.spring_layout(graph_nx)  # positions for all nodes
        nx.draw(graph_nx, pos, with_labels=True)
        labels = nx.get_edge_attributes(graph_nx, 'key')
        nx.draw_networkx_edge_labels(graph_nx, pos, edge_labels=labels)
        plt.show()

    
class AdapterBlock(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(AdapterBlock, self).__init__()
        self.layer_norm = nn.LayerNorm(input_dim, eps=1e-6)
        self.gcn = GCNConv(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc = nn.Linear(hidden_dim, input_dim)  # Restore dimension to input_dim
        self.res_fc = nn.Linear(input_dim, input_dim)  # Transform the residual

    def forward(self, x, edge_index):
        x_res = self.res_fc(x)  # Transform the residual tensor
        x = self.layer_norm(x)
        x = self.gcn(x, edge_index)
        x = self.relu(x)
        x = self.fc(x)
        return x + x_res  # Now you can add the tensors


from transformers import T5ForConditionalGeneration

class TransformerGCN(nn.Module):
    def __init__(self, vocab_size, adapter_dim):
        super(TransformerGCN, self).__init__()
        self.transformer = T5ForConditionalGeneration.from_pretrained(model_name)
        self.hidden_size = self.transformer.config.hidden_size  # Get the hidden size from the config
        self.reduce_dim = nn.Linear(32128, adapter_dim)  # Use the hidden size instead of 768

        # Freeze the parameters of the T5 model
        for param in self.transformer.parameters():
            param.requires_grad = False

        self.adapter_blocks = nn.ModuleList([
        AdapterBlock(block.layer[1].DenseReluDense.wo.weight.size(0), adapter_dim) for block in self.transformer.encoder.block
        ])

        self.output_head = nn.Linear(adapter_dim, vocab_size)

    def forward(self, input_ids, attention_mask, edge_index):
        if input_ids.dim() == 1:  # If the input is 1D (batch size 1)
            input_ids = input_ids.unsqueeze(0)  # Add a batch dimension
        if attention_mask.dim() == 1:  # Same for the attention_mask
            attention_mask = attention_mask.unsqueeze(0)

        shifted_input_ids = torch.cat([torch.zeros((input_ids.size(0), 1), dtype=torch.long, device=input_ids.device), input_ids[:, :-1]], dim=-1)

        input_embeds = self.transformer.get_input_embeddings()(input_ids)
        hidden_states = input_embeds
        for block, adapter_block in zip(self.transformer.encoder.block, self.adapter_blocks):
            hidden_states, _ = block(hidden_states, attention_mask=attention_mask, encoder_hidden_states=None, encoder_attention_mask=None)
            hidden_states = adapter_block(hidden_states, edge_index)

        transformer_outputs = self.transformer(inputs_embeds=hidden_states, attention_mask=attention_mask, decoder_input_ids=shifted_input_ids)
        transformer_outputs = self.reduce_dim(transformer_outputs[0])
        return self.output_head(transformer_outputs)


class ModifiedT5Block(nn.Module):
    def __init__(self, original_block, adapter_dim):
        super(ModifiedT5Block, self).__init__()
        self.original_block = original_block
        self.adapter = AdapterBlock(original_block.layer[1].DenseReluDense.wi.weight.size(-1), adapter_dim)

    def forward(self, x, edge_index, **kwargs):
        x, _ = self.original_block(x, **kwargs)
        return self.adapter(x, edge_index)



from torch_geometric.data import DataLoader as GeometricDataLoader

def train(model, dataloader, epochs, device):
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001) # Lower learning rate
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min') # Add learning rate scheduler
    tokenizer = T5TokenizerFast.from_pretrained(model_name)
    criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)

    for epoch in range(epochs):
        model.train()
        i = 0
        for data in tqdm(dataloader):
            data = data.to(device) # Moving batch to device
            optimizer.zero_grad()

            outputs = model(input_ids=data.x, attention_mask=data.attention_mask, edge_index=data.edge_index)
            loss = criterion(outputs.view(-1, outputs.size(-1)), data.y.view(-1))
            loss.backward()
            optimizer.step()
            
            if i % 100 == 0:
                print(f"Epoch: {epoch}, Iteration: {i}, Loss: {loss.item()}")
            
            i += 1 

# Usage
dataset_dict = load_dataset('web_nlg', 'webnlg_challenge_2017')['train']
dataset = WebNLGDataset(dataset_dict)
vocab_size = len(dataset.tokenizer)
model = TransformerGCN(vocab_size=vocab_size, adapter_dim=512)
dataloader = GeometricDataLoader(dataset, batch_size=2)
train(model, dataloader, epochs=2, device=torch.device('cuda'))


/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

Generating train split:   0%|          | 0/6940 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/4615 [00:00<?, ? examples/s]

Dataset web_nlg downloaded and prepared to /root/.cache/huggingface/datasets/web_nlg/webnlg_challenge_2017/0.0.0/28ffb892f7f42450dd9558684aa43bcaf44b1b3bf0d77cb8d73534646af88dda. Subsequent calls will reuse this data.


...

 63%|██████▎   | 2202/3470 [07:09<04:02,  5.23it/s]

Epoch: 1, Iteration: 2200, Loss: 3.874729633331299


 66%|██████▋   | 2302/3470 [07:28<03:49,  5.09it/s]

Epoch: 1, Iteration: 2300, Loss: 3.923366069793701


 68%|██████▊   | 2356/3470 [07:39<03:36,  5.15it/s]

In [3]:
def test(model, dataloader, device):
    model = model.to(device)
    model.eval()

    with torch.no_grad():
        for data in dataloader:
            data = data.to(device) 

            data.x = data.x.unsqueeze(0)  # Add batch dimension
            data.attention_mask = data.attention_mask.unsqueeze(0) 

            outputs = model.transformer.generate(input_ids=data.x, attention_mask=data.attention_mask, decoder_start_token_id=model.transformer.config.pad_token_id)

            # Convert the tensor outputs to text using the tokenizer
            output_text = [dataset.tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in outputs]

            # print input and output
            print(f"Input: {dataset.tokenizer.decode(data.x[0].tolist(), skip_special_tokens=True)}")  # Convert tensor to list
            print(f"Output: {output_text}")

# Usage
dataset_dict = load_dataset('web_nlg', 'webnlg_challenge_2017')['test']
dataset_dict = [sample for sample in dataset_dict if sample['lex']['text']] # filter out samples with empty targets 
dataset = WebNLGDataset(dataset_dict)
dataloader = GeometricDataLoader(dataset, batch_size=5)
test(model, dataloader, device=torch.device('cuda'))


  0%|          | 0/3 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Input: Aaron S Daggett was awarded the Purple Heart. The Battle of Mine Run was one fought by Aaron S Daggett. Stellendam, Netherlands is the birthplace of Ab Klink. Abdul Rahman Ya'kub was in office while Tuanku Bujang Tuanku Othman was Vice President. Abdul Taib Mahmud belongs to the party of Parti Bumiputera Sarawak.
Output: ["Ab Klink wurde von Abdul Rahman Ya'kub in Amt angewart"]
Input: Abdul Taib Mahmud's successor was Sulaiman Abdul Rahman Taib. Abdulsalami Abubakar ended his career on 1999-05-29. Abdulsalami Abubakar was born in Minna. Abdulsalami Abubakar's birthplace was Niger State. Abner W. Sibal ended his military career January 3, 1965.
Output: ['Abdulsalami Abubakar hat seine militärische Karriere Ende Ende 1999']
Input: Abner W Sibal died in Alexandria, Virginia. Adam Holloway was born in Kent. Adam Holloway's residence is Gravesend. The alma mater of Adenan Satem is the University of Adelaide. Adolf Schärf's place of birth was Mikulov.
Output: ['Adam Holloway wurde in

KeyboardInterrupt: 

In [8]:
!pip install sacrebleu 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 5.0 MB/s eta 0:00:00


In [33]:
from sacrebleu import corpus_bleu
from random import sample
from tqdm import tqdm
from torch_geometric.data import DataLoader as GeometricDataLoader

# load the WebNLG validation dataset
validation_dataset = load_dataset('web_nlg', 'webnlg_challenge_2017')['test']
validation_dataset = [sample for sample in validation_dataset if sample['lex']['text']]  # filter out samples with empty targets

validation_data = WebNLGDataset(validation_dataset)

# set up the validation data loader
validation_loader = GeometricDataLoader(validation_data, batch_size=1, shuffle=False)

# switch model to evaluation mode
model.eval()

device = 'cuda'

# generate predictions for the validation dataset
predictions = []
references = []
with torch.no_grad():
    for data in tqdm(validation_loader, desc='Validation Progress', leave=False):
        data.x = data.x.to(device).unsqueeze(0)  # add batch dimension
        data.attention_mask = data.attention_mask.to(device).unsqueeze(0)  # add batch dimension

        outputs = model.transformer.generate(input_ids=data.x, attention_mask=data.attention_mask, decoder_start_token_id=model.transformer.config.pad_token_id)
        # convert token IDs to strings
        predicted_texts = dataset.tokenizer.batch_decode(outputs, skip_special_tokens=True)
        target_texts = dataset.tokenizer.batch_decode(data.y, skip_special_tokens=True)
        # append predicted and target texts for BLEU evaluation
        predictions.extend(predicted_texts)
        references.extend(target_texts)


  0%|          | 0/3 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)
Validation Progress:   0%|          | 0/2753 [00:00<?, ?it/s]/opt/con

In [35]:
# calculate BLEU scores
#bleu = corpus_bleu(predictions, [references])

multiple_references = []
for i in range(len(validation_dataset)):
    multiple_references.append(validation_dataset[i]['lex']['text'])
    
#bleu = corpus_bleu(predictions, references)
bleu_multiple = corpus_bleu(predictions, multiple_references)

#print(f"BLEU score: {bleu.score}")
print(f"BLEU score with multiple references: {bleu_multiple.score}")

BLEU score with multiple references: 100.00000000000004


In [36]:
len(predictions)

2753

In [37]:
len(validation_dataset)

2753

In [40]:
i=1860
print(validation_dataset[i])
print('------------------------')
print(validation_dataset[i]['original_triple_sets']['otriple_set'])
print('------------------------')
print(predictions[i])
print('------------------------')
print(multiple_references[i])

{'category': 'Astronaut', 'size': 7, 'eid': 'Id970', 'original_triple_sets': {'otriple_set': [['William_Anders | dateOfRet | "1969-09-01"^^xsd:date', 'William_Anders | mission | Apollo_8', 'William_Anders | nationality | United_States', 'William_Anders | birthPlace | British_Hong_Kong', 'Apollo_8 | crew2Up | Buzz_Aldrin', 'Apollo_8 | crewMembers | Frank_Borman', 'Apollo_8 | operator | NASA']]}, 'modified_triple_sets': {'mtriple_set': [['William_Anders | dateOfRetirement | "1969-09-01"', 'William_Anders | was a crew member of | Apollo_8', 'William_Anders | nationality | United_States', 'William_Anders | birthPlace | British_Hong_Kong', 'Apollo_8 | backup pilot | Buzz_Aldrin', 'Apollo_8 | crewMembers | Frank_Borman', 'Apollo_8 | operator | NASA']]}, 'shape': '', 'shape_type': '', 'lex': {'comment': ['good', 'good', 'good'], 'lid': ['Id1', 'Id2', 'Id3'], 'text': ["William Anders was born in British Hong Kong and is a U.S Citizen. William was a member of the Apollo 8 crew (along with Frank

In [ ]:
# Print the postprocessed output for the first item in the dataset
dataset[-3]